In [2]:
import pandas as pd

Создадим пустой daraframe

In [3]:
data = []
df = pd.DataFrame(data, columns=['id', 'Город_дистрибьютора','Торг_точка_грязная','Торг_точка_грязная_адрес','outlet_clean_id'])

распарсим данные из файла в dataframe

In [5]:
f = open('outlets.txt', 'r')
for line in f:
    if line.startswith("("):
        line = line.replace("(","")
        line = line.replace(")","")
        line = line[:-1]
        data = line.split('\' ')
        data = data[0]
        l = ['"{}"'.format(s) for s in data.split('\'') if s not in ('', ', ')]
        l = [elem.replace(",","").replace("\"","").strip() for elem in l]
        # break
        # print(l)
        dict = {'id': l[0], 'Город_дистрибьютора': l[1], 'Торг_точка_грязная': l[2],'Торг_точка_грязная_адрес':l[3],'outlet_clean_id':l[4]}
        df = df.append(dict,  ignore_index = True)



In [5]:
df.head()

,id,Город_дистрибьютора,Торг_точка_грязная,Торг_точка_грязная_адрес,outlet_clean_id
0,1,Ростов-на-Дону,ИП Гаврилов С.А.,п.Глубокий ул.Артёма 44,NULL
1,2,Ростов-на-Дону,ИП Малышева,-,NULL
2,3,Ростов-на-Дону,Абасян А.П. ИП,он же,NULL
3,4,Ростов-на-Дону,Азатян НС. ИП ул. Ленина,он же,NULL
4,5,Ростов-на-Дону,Айдинов В.Я.,он же,NULL
...,...,...,...,...,...
20203,20204,Ростов-на-Дону,Яровой А.Т.ИП,346535 Ростовская обл Шахты г Кошевого ул дом ...,NULL
20204,20205,Ростов-на-Дону,Яровой В.Т. ИП м-н \Весна\,-,NULL
20205,20206,Ростов-на-Дону,Ярута С.М. ИП,347332 Ростовская область Волгодонской р-н х. ...,NULL
20206,20207,Ростов-на-Дону,Яхонов Б.А. ИП,346380 Ростовская обл Красносулинский р-н Лихо...,NULL


Удалим колонку 'Город_дистрибьютора', т.к. значение в ней у всех одинаковое

In [6]:
df = df.drop(columns=['Город_дистрибьютора'])

In [ ]:
df.head()

,id,Торг_точка_грязная,Торг_точка_грязная_адрес,outlet_clean_id
0,1,ИП Гаврилов С.А.,п.Глубокий ул.Артёма 44,NULL
1,2,ИП Малышева,-,NULL
2,3,Абасян А.П. ИП,он же,NULL
3,4,Азатян НС. ИП ул. Ленина,он же,NULL
4,5,Айдинов В.Я.,он же,NULL


In [7]:
df['addr'] = df['Торг_точка_грязная']+df['Торг_точка_грязная_адрес']

In [8]:
df.head()

,id,Торг_точка_грязная,Торг_точка_грязная_адрес,outlet_clean_id,addr
0,1,ИП Гаврилов С.А.,п.Глубокий ул.Артёма 44,NULL,ИП Гаврилов С.А.п.Глубокий ул.Артёма 44
1,2,ИП Малышева,-,NULL,ИП Малышева-
2,3,Абасян А.П. ИП,он же,NULL,Абасян А.П. ИПон же
3,4,Азатян НС. ИП ул. Ленина,он же,NULL,Азатян НС. ИП ул. Ленинаон же
4,5,Айдинов В.Я.,он же,NULL,Айдинов В.Я.он же


Реализация препроцессинга текста: удаление спец символов, стоп-слов,приведение к нижнему регистру

In [9]:
import regex as re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = stopwords.words('english')
STOPWORDS = set(STOPWORDS)
    
def text_prepare(text, STOPWORDS):
    """
        text: a string
        
        return: a clean string
    """
    REPLACE_BY_SPACE_RE = re.compile('[\n\"\'/(){}\[\]\|@,;#]')
    text = re.sub(REPLACE_BY_SPACE_RE, ' ', text)
    text = re.sub(' +', ' ', text)
    text = text.lower()

    # delete stopwords from text
    text = ' '.join([word for word in text.split() if word not in STOPWORDS]) 
    text = text.strip()
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
df['addr'] = df['addr'].apply(lambda x: text_prepare(x,STOPWORDS))

In [11]:
df.head()

,id,Торг_точка_грязная,Торг_точка_грязная_адрес,outlet_clean_id,addr
0,1,ИП Гаврилов С.А.,п.Глубокий ул.Артёма 44,NULL,ип гаврилов с.а.п.глубокий ул.артёма 44
1,2,ИП Малышева,-,NULL,ип малышева-
2,3,Абасян А.П. ИП,он же,NULL,абасян а.п. ипон же
3,4,Азатян НС. ИП ул. Ленина,он же,NULL,азатян нс. ип ул. ленинаон же
4,5,Айдинов В.Я.,он же,NULL,айдинов в.я.он же


In [12]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
def get_similarity_matrix(df, threshold):
    """
    Функция, соединяющая функции для получения топков

    Parameters
    ----------
    df : DataFrame
        Таблица с центрами, топ словами, векторным представлением и весами слов для каждого топика
    threshold : float
        трешхолд для отбора похожих топиков

    Returns
    -------
    df : DataFrame
        Таблица со значениями косинусных расстояний между центрами топиков
    similar_topics : list
        Список топиков, похожесть которых выше указанного порога
    """
    matrix = []
    similar_topics = []
    for i in range(len(df['centers'])):
        row = []
        for j in range(len(df['centers'])):
            dist = cosine_similarity(df['centers'][i], df['centers'][j])  # cosine_similarity
            row.append(dist[0][0])
            if j > i and dist[0][0] > threshold:
                similar_topics.append((df['id'][i], df['id'][j]))
        matrix.append(row)

    df = pd.DataFrame(matrix)
    return df, similar_topics

def get_center(top_feat):
    """
    Функция, находящая центр одного топика методом kmeans

    Parameters
    ----------
    top_feat : list
        Массив топ признаков топика
    Returns
    -------
    list
        Центр кластера
    """
    kmeans = KMeans(n_clusters=1)
    kmeans.fit(top_feat)
    kmeans.predict(top_feat)
    return kmeans.cluster_centers_

In [13]:
df.head()

,id,Торг_точка_грязная,Торг_точка_грязная_адрес,outlet_clean_id,addr
0,1,ИП Гаврилов С.А.,п.Глубокий ул.Артёма 44,NULL,ип гаврилов с.а.п.глубокий ул.артёма 44
1,2,ИП Малышева,-,NULL,ип малышева-
2,3,Абасян А.П. ИП,он же,NULL,абасян а.п. ипон же
3,4,Азатян НС. ИП ул. Ленина,он же,NULL,азатян нс. ип ул. ленинаон же
4,5,Айдинов В.Я.,он же,NULL,айдинов в.я.он же


Создание нового dataframe, чтобы при нахождении похожих топиков, удалять не из исходого датасета, а из нового (в последствии этот датасет и окажется таблицей `outlets_clean`)

In [14]:
new_df = df[['id','addr','outlet_clean_id']]

In [15]:
new_df.head()

,id,addr,outlet_clean_id
0,1,ип гаврилов с.а.п.глубокий ул.артёма 44,NULL
1,2,ип малышева-,NULL
2,3,абасян а.п. ипон же,NULL
3,4,азатян нс. ип ул. ленинаон же,NULL
4,5,айдинов в.я.он же,NULL


Векторизуем тексты наиболее популярной и пройстой векторизацией tf-idf

In [18]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2), max_df=0.9, min_df=5, token_pattern='(\S+)')
X = tfidf_vectorizer.fit_transform(new_df['addr'])
X1 = X.toarray()
# new_df['transformed'] = X1

In [73]:
new_df.tail()

,id,addr,outlet_clean_id,centers
20203,20204,яровой а.т.ип346535 ростовская обл шахты г кош...,20204,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
20204,20205,яровой в.т. ип м-н \весна\-,20205,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
20205,20206,ярута с.м. ип347332 ростовская область волгодо...,20206,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
20206,20207,яхонов б.а. ип346380 ростовская обл красносули...,20207,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
20207,20208,яцкевич е.ю.ип344002 ростовская обл ростов-на-...,20208,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


Сопоставим каждому тексту его векторизованное представление в datafrme

In [19]:
centers = []
for item in X1:
    centers.append(item.reshape(1,-1))

new_df['centers'] = centers


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


В следующем блоке кода реализуется нахождение дубликатов (похожих) адресов

Для этого ранее была реализована функция `get_similarity_matrix`, которая высчитывает косинусное расстояние между векторами, на основе полученных результатов, задаем кластер, если был у первого, то и второму задаем тот же кластер, если у первого нет, но у второго есть, то первому задаем класс второго, если ни у кого нет, то создаем новый посредством постепенного увеличения переменной `id`

In [ ]:
similarity_matrix, similar_topics = get_similarity_matrix(new_df, 0.7)
id = 1
while len(similar_topics) > 0:
    # дать номер кластера двум адресам одинаковый
    i = similar_topics[0][0]
    j = similar_topics[0][1]
    if df[df['id']==i]['outlet_clean_id'] != "NULL":
        df[df['id']==j]['outlet_clean_id'] = df[df['id']==i]['outlet_clean_id']
        new_df[new_df['id']==i]['outlet_clean_id'] = df[df['id']==i]['outlet_clean_id']
    elif df[df['id']==j]['outlet_clean_id'] != "NULL":
        df[df['id']==i]['outlet_clean_id'] = df[df['id']==j]['outlet_clean_id']
        new_df[new_df['id']==i]['outlet_clean_id'] = df[df['id']==j]['outlet_clean_id']
    else:
        df[df['id']==j]['outlet_clean_id'] = id
        df[df['id']==i]['outlet_clean_id'] = id
        new_df[new_df['id']==i]['outlet_clean_id'] = id
        id+=1
    new_df = new_df.drop(new_df[new_df['id'] == j].index)

    similarity_matrix, similar_topics = get_similarity_matrix(new_df, 0.7)

Для итоговых таблиц необходимо удалить из датасета df колонку addr, а из датасета new_df колонки `id`,`centers`,а также переименовать `outlet_clean_id` в `id`

In [ ]:
new_df = new_df.drop(columns = ["id","centers"])
new_df.rename(columns={"outlet_clean_id": "id"})
df = df.drop(columns = ['addr'])

Сохраним полученные результаты в файлы

In [ ]:
df.to_csv("outlets",index=False)
new_df.to_csv("outlets_clean",index=False)